# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame. WeatherDataCities.csv
weather_citi = "../output_data/WeatherDataCities.csv"

# # Read WeatherDataCities.csv and store into Pandas DataFrames
weather_cities = pd.read_csv(weather_citi)

weather_cities.head()

,City,Lat,Lng,Max Temp (K),Humidity,Cloudiness,Country,Wind Speed,Date
0,Liverpool,53.41,-2.98,274.26,85,100,GB,1.00,1609204716
1,Severomorsk,69.07,33.42,266.15,79,75,RU,2.00,1609204995
2,Narsaq,60.92,-46.05,272.15,68,29,GL,3.60,1609204997
3,Kumul,42.80,93.45,261.64,75,0,CN,2.29,1609204998
4,Butaritari,3.07,172.79,300.68,79,100,KI,9.07,1609205000


In [13]:
weather_cities.dtypes

City             object
Lat             float64
Lng             float64
Max Temp (K)    float64
Humidity          int64
Cloudiness        int64
Country          object
Wind Speed      float64
Date              int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [14]:
# Store 'Lat' and 'Lng' into  locations as floats
locations = weather_cities[["Lat", "Lng"]].astype(float)
weights = weather_cities["Humidity"].astype(float)

In [15]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph and zero cloudiness.
fitting_weather = weather_cities.loc[(weather_cities['Humidity']>70) &
                                     (weather_cities['Humidity']<80) &
                                     (weather_cities['Wind Speed']<10) &
                                     (weather_cities['Cloudiness']==0)]

fitting_weather.shape[0]

24

In [27]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
fit_weather = fitting_weather.dropna()
fit_weather

,City,Lat,Lng,Max Temp (K),Humidity,Cloudiness,Country,Wind Speed,Date
3,Kumul,42.80,93.45,261.64,75,0,CN,2.29,1609204998
15,Karpathos,35.51,27.21,288.15,77,0,GR,3.10,1609205013
25,Punta Arenas,-53.15,-70.92,283.15,71,0,CL,3.60,1609204967
76,Safaga,26.73,33.94,289.15,72,0,EG,4.10,1609205089
96,Chlórakas,34.80,32.41,286.48,71,0,CY,4.10,1609205119
117,Atuona,-9.80,-139.03,299.25,73,0,PF,6.86,1609205147
139,Guerrero Negro,27.98,-114.06,287.95,74,0,MX,4.91,1609205176
170,Tura,25.52,90.22,286.10,76,0,IN,1.09,1609205216
201,Pushkar,26.50,74.55,275.33,77,0,IN,1.05,1609205256
214,Aswān,24.09,32.91,289.10,72,0,EG,3.63,1609205042


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# Store into variable named hotel_df.
hotel_df = fit_weather
hotel_name = ""

# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = hotel_name

hotel_df

,City,Lat,Lng,Max Temp (K),Humidity,Cloudiness,Country,Wind Speed,Date,Hotel Name
3,Kumul,42.80,93.45,261.64,75,0,CN,2.29,1609204998,
15,Karpathos,35.51,27.21,288.15,77,0,GR,3.10,1609205013,
25,Punta Arenas,-53.15,-70.92,283.15,71,0,CL,3.60,1609204967,
76,Safaga,26.73,33.94,289.15,72,0,EG,4.10,1609205089,
96,Chlórakas,34.80,32.41,286.48,71,0,CY,4.10,1609205119,
117,Atuona,-9.80,-139.03,299.25,73,0,PF,6.86,1609205147,
139,Guerrero Negro,27.98,-114.06,287.95,74,0,MX,4.91,1609205176,
170,Tura,25.52,90.22,286.10,76,0,IN,1.09,1609205216,
201,Pushkar,26.50,74.55,275.33,77,0,IN,1.05,1609205256,
214,Aswān,24.09,32.91,289.10,72,0,EG,3.63,1609205042,


In [38]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
params = {"key": g_key}
hotel = []
telly = 0

for city in hotel_df['City']:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    set_coordinates = str(hotel_df['Lat']) , str(hotel_df['Lng'])
    set_radius = 5000
    target_type = 'hotel'
    params = {
    "location": set_coordinates,
    "radius": set_radius,
    "type": target_type,
    "key": g_key,
    telly += 1
    response = requests.get(base_url, params=params)
    hotel_results = response.json()
    hotels.append(hotel_results['results'][0]['name'])

SyntaxError: invalid syntax (<ipython-input-38-1d0505c6ce12>, line 19)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(Hotel Name)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig